In [ ]:
from modelscope import snapshot_download
snapshot_download('iic/CosyVoice2-0.5B', local_dir='/obs/xuke/cosyvoicepretrained_models/CosyVoice2-0.5B')
snapshot_download('iic/CosyVoice-300M', local_dir='/obs/xuke/cosyvoicepretrained_models/CosyVoice-300M')
snapshot_download('iic/CosyVoice-300M-SFT', local_dir='/obs/xuke/cosyvoicepretrained_models/CosyVoice-300M-SFT')
snapshot_download('iic/CosyVoice-300M-Instruct', local_dir='/obs/xuke/cosyvoicepretrained_models/CosyVoice-300M-Instruct')
snapshot_download('iic/CosyVoice-ttsfrd', local_dir='/obs/xuke/cosyvoicepretrained_models/CosyVoice-ttsfrd')

In [ ]:
import sys
sys.path.append('third_party/Matcha-TTS')
from cosyvoice.cli.cosyvoice import CosyVoice, CosyVoice2
from cosyvoice.utils.file_utils import load_wav
import torchaudio
import json
from datasets import load_from_disk

In [ ]:
import os
os.environ["MODELSCOPE_CACHE"] = "/obs/xuke/modelscope_cache"
cosyvoice = CosyVoice2('/obs/xuke/cosyvoicepretrained_models/CosyVoice2-0.5B', load_jit=False, load_trt=False, load_vllm=False, fp16=False)


In [ ]:
# 加载说话人
speaker_base_path = "/obs/xuke/audio_source"
with open(f"{speaker_base_path}/real_audio.json","r",encoding="utf-8") as f:
    real_audio = json.load(f)

## 加载数据集

In [ ]:
# ds = load_from_disk("/obs/xuke/smart-turn/llm_gen_topic_cn")
seg_ds = load_from_disk("/obs/xuke/smart-turn/test_seg_tts_50")

In [ ]:
import random

save_path = "/obs/xuke/cosyvoice/outputs"
for index,item in enumerate(seg_ds):
    speaker = random.choice(real_audio)
    prompt_speech_16k = load_wav(f"{speaker_base_path}/{speaker['filename']}", 16000)
    if index > 50:
        break
    seg_text = item['segmented']
    ori_text = item['original']
    print(f"{index}")
    for i, j in enumerate(cosyvoice.inference_zero_shot(seg_text, speaker["text"], prompt_speech_16k, stream=False)):
        torchaudio.save(f'{save_path}/zero_shot_{index}_{i}_seg.wav', j['tts_speech'], cosyvoice.sample_rate)
    for i, j in enumerate(cosyvoice.inference_zero_shot(ori_text, speaker["text"], prompt_speech_16k, stream=False)):
        torchaudio.save(f'{save_path}/zero_shot_{index}_{i}_ori.wav', j['tts_speech'], cosyvoice.sample_rate)